In [1]:
! pip install mysql-connector-python
import mysql.connector
import pandas as pd
import json

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
con = open('ref_con/db-prima', 'r').read().splitlines()
how = input('how to connect(local/public):')

mydb = mysql.connector.connect(
  host=con[0] if how == 'public' else con[1],
  user=con[2],
  password=con[3],
  database=con[4] 
)

mycursor = mydb.cursor(dictionary=True)

In [104]:
def exec_q(qT):
  mycursor.execute(qT)
  myresult = mycursor.fetchall()
  jD = []
  for x in myresult:
    jD.append(x['json_result'])
  JReturn = '['+','.join(jD)+']'
  return JReturn

def exec_to_result(q):
  mycursor.execute(q)
  return mycursor.fetchall()

def get_erm_result(code,v='',limit=''):
    verQ = ' TRUE ' if v=='' else ' versi = "'+v+'" '
    limQ = '' if limit=='' else ' LIMIT '+limit 
    return exec_q("select json_result from primaprod.mr_result inner join primaprod.mr_form on \
id_form=mr_form.id where kode like '"+code+"' AND "+verQ+limQ)

def get_erm_result_from_RM(rm):
  #dapatkan barisan DR
  rawPKP = exec_to_result("select poli_kunjungan_pasien.attr_tambahan from poli_kunjungan_pasien inner join pendaftaran\
    using (no_reg) where pendaftaran.no_rm = '"+rm+"' and poli_kunjungan_pasien.status_ok = 1\
      and attr_tambahan like '%\"emr\":%'")
  return get_attr_emr(rawPKP)

def get_erm_result_from_NOREG(nr):
  #dapatkan barisan attr
  rawPKP = exec_to_result("select attr_tambahan from poli_kunjungan_pasien where no_reg = '"+nr+"' and status_ok = 1\
     and attr_tambahan like '%\"emr\":%'")
  return get_attr_emr(rawPKP)

def get_erm_result_from_DR(dr):
  rawPKP = exec_to_result("select attr_tambahan from poli_kunjungan_pasien where id = '"+dr+"' and status_ok = 1\
    and attr_tambahan like '%\"emr\":%'")
  return get_attr_emr(rawPKP)

def get_attr_emr(attrs):
  EMRid = []
  for i in attrs:
    attr = json.loads(i['attr_tambahan'])
    for j in attr['emr']:
      EMRid.append(j)
  mycursor.execute('select json_result,kode from primaprod.mr_result inner join primaprod.mr_form on \
id_form=mr_form.id where mr_result.id in ("'+'","'.join([str(int) for int in EMRid])+'")')
  return parse_code_to_key(mycursor.fetchall()) 

def parse_code_to_key(dic):
  rDic = {}
  for i in dic:
    jR = json.loads(i['json_result'])
    for k in jR:
      rDic[i['kode']+'.'+k] = []
      rDic[i['kode']+'.'+k].append(jR[k])
  return rDic


In [21]:
DFresult = pd.read_json(get_erm_result('RM-B01','','100'))

In [ ]:
DFresult

In [ ]:
get_erm_result_from_RM('2586113')

In [108]:
df = pd.DataFrame(get_erm_result_from_NOREG('2205170889'))
df

,RM-B02.kasus,RM-B02.informasi_dari,RM-B02.cara_masuk,RM-B02.cara_masuk_dengan_bantuan,RM-B02.gangguan_perilaku,RM-B02.sifat_gangguan_perilaku,RM-B02.status_alergi,RM-B02.alergi,RM-B02.orang_terdekat_dengan_pasien,RM-B02.pembuat_keputusan,...,RM-I02.dosis,RM-I02.rute,RM-I02.frekwensi,RM-I02.dosis_terakhir,RM-I02.keterangan,RM-I02.instruksi_dokter_saat_masuk_rumah_sakit,RM-I02.instruksi_dokter_saat_keluar_rumah_sakit,RM-I02.user_ttd_dpjp,RM-I02.user_ttd_apoteker,RM-I02.user_ttd_perawat
0,Non KLL,Auto-anamnesis,Kursi Roda,,Tidak Terganggu,Tidak Membahayakan,Tidak,,keluarga,keluarga,...,20MG && -,Oral && Oral,2X1 && K/P,17-05-2022 && 17-05-2022,&&,&&,&&,,Usi E,
